<a href="https://colab.research.google.com/github/umututku03/3D-Rendering-Camera-Rotation-Torch/blob/main/3D_Rendering_Camera_Rot_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import shutil
import numpy as np
from PIL import Image, ImageDraw
import random
import torch
from torch import nn
from math import sin, cos, radians
import matplotlib.pyplot as plt

In [ ]:
# Ensure reproducibility
torch.manual_seed(14)
np.random.seed(14)
random.seed(14)

In [ ]:
# Define the cube vertices and edges
cube_vertices = np.array([
    [1, 1, -1], [-1, 1, -1], [-1, -1, -1], [1, -1, -1],
    [1, 1, 1], [-1, 1, 1], [-1, -1, 1], [1, -1, 1]
])

cube_edges = [
    (0, 1), (1, 2), (2, 3), (3, 0),
    (4, 5), (5, 6), (6, 7), (7, 4),
    (0, 4), (1, 5), (2, 6), (3, 7)
]

In [ ]:
class LookAt(nn.Module):
    """
    A neural network module that constructs a view matrix for transforming
    coordinates from one position to another, emulating the behavior of a camera
    looking from one position to another.
    """
    def __init__(self):
        """
        Initializes the LookAt module by calling the constructor of the parent class nn.Module.
        """
        super(LookAt, self).__init__()

    def forward(self, from_pos, to_pos, up):
        """
        Constructs the view matrix given the position to look from, the position
        to look at, and the up direction vector.
        """
        forward = from_pos - to_pos
        forward = forward / torch.norm(forward)
        right = torch.cross(up, forward)
        right = right / torch.norm(right)
        up = torch.cross(forward, right)
        view_matrix = torch.eye(4)
        view_matrix[0, :3] = right
        view_matrix[1, :3] = up
        view_matrix[2, :3] = forward
        view_matrix[:3, 3] = -torch.matmul(view_matrix[:3, :3], from_pos.unsqueeze(1)).squeeze()
        return view_matrix


In [ ]:
class ProjectVertex(nn.Module):
    """
    A neural network module that projects a 3D vertex into 2D space using a given projection matrix.
    """
    def __init__(self, projection_matrix):
        """
        Initializes the ProjectVertex module with a given projection matrix.
        """
        super(ProjectVertex, self).__init__()
        self.projection_matrix = projection_matrix

    def forward(self, vertex, view_matrix):
        """
        Projects the given 3D vertex into 2D space using the view and projection matrices.
        """
        vertex_homogeneous = torch.cat((vertex, torch.ones(1)), dim=0)
        transformed_vertex = view_matrix @ vertex_homogeneous
        projected_vertex = self.projection_matrix @ transformed_vertex
        projected_vertex = projected_vertex[:2] / projected_vertex[3]
        return projected_vertex


In [ ]:
# Initial camera setup
target_pos = torch.tensor([0.0, 0.0, 0.0], dtype=torch.float32)
up_vector = torch.tensor([0.0, 1.0, 0.0], dtype=torch.float32)

# Projection matrix setup (assuming perspective projection)
fov = 60  # Field of view in degrees
aspect_ratio = 1.0  # Aspect ratio
near = 1.0  # Near clipping plane
far = 100.0  # Far clipping plane
fov_rad = np.radians(fov)
f = 1 / np.tan(fov_rad / 2)
projection_matrix = torch.tensor([
    [f / aspect_ratio, 0, 0, 0],
    [0, f, 0, 0],
    [0, 0, (far + near) / (near - far), (2 * far * near) / (near - far)],
    [0, 0, -1, 0]
], dtype=torch.float32)

In [ ]:
look_at = LookAt()
project_vertex = ProjectVertex(projection_matrix)

In [ ]:
# Rendering parameters
r = 5
num_steps = 100

In [ ]:
# Ensure the output directories exist
render_dir = "renders_pytorch"
extrinsic_dir = "extrinsics"
points_dir = "computed_2d_points"

for directory in [render_dir, extrinsic_dir, points_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)

In [ ]:
# Image rendering loop
for step in range(num_steps):
    alpha = step * 2 * np.pi / num_steps
    camera_pos = torch.tensor([
        target_pos[0] + r * np.cos(alpha),  # x-coordinate
        target_pos[1] - 3.5,                # y-coordinate (elevation) kept constant
        target_pos[2] + r * np.sin(alpha)   # z-coordinate
    ], dtype=torch.float32)

    view_matrix = look_at(camera_pos, target_pos, up_vector)

    # Save the extrinsic matrix
    np.savetxt(f"{extrinsic_dir}/extrinsic_{step}.txt", view_matrix.detach().numpy())

    image_size = 500
    image = Image.new("RGB", (image_size, image_size), "white")
    draw = ImageDraw.Draw(image)

    # Project and draw each edge of the cube
    points_2d = []
    for edge in cube_edges:
        v0, v1 = cube_vertices[edge[0]], cube_vertices[edge[1]]
        p0 = project_vertex(torch.tensor(v0, dtype=torch.float32), view_matrix)
        p1 = project_vertex(torch.tensor(v1, dtype=torch.float32), view_matrix)
        p0 = (image_size / 2 * (p0 + 1)).int().numpy()
        p1 = (image_size / 2 * (p1 + 1)).int().numpy()
        draw.line([tuple(p0), tuple(p1)], fill="black")
        points_2d.append(p0)
        points_2d.append(p1)

    # Save the 2D points
    points_2d = np.unique(np.array(points_2d), axis=0)  # Remove duplicate points
    np.savetxt(f"{points_dir}/points_{step}.txt", points_2d, fmt='%d')

    # Save the image
    filename = f'{render_dir}/frame_{step}.png'
    image.save(filename)

print(f'Rendering completed. Images saved in {render_dir}, {extrinsic_dir}, and {points_dir} directories.')

In [ ]:
# Ensure the output directories exist in the current working directory
output_dir_frames = "/kaggle/working/cube_images_fixed_frames"
output_dir_extrinsics = "/kaggle/working/cube_images_fixed_extrinsics"
output_dir_points = "/kaggle/working/cube_images_fixed_points"

for directory in [output_dir_frames, output_dir_extrinsics, output_dir_points]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# Copy images and data to the output directories
for file in os.listdir(render_dir):
    shutil.copy(os.path.join(render_dir, file), output_dir_frames)

for file in os.listdir(extrinsic_dir):
    shutil.copy(os.path.join(extrinsic_dir, file), output_dir_extrinsics)

for file in os.listdir(points_dir):
    shutil.copy(os.path.join(points_dir, file), output_dir_points)

# Create zip files
shutil.make_archive(output_dir_frames, 'zip', output_dir_frames)
shutil.make_archive(output_dir_extrinsics, 'zip', output_dir_extrinsics)
shutil.make_archive(output_dir_points, 'zip', output_dir_points)

print(f'Images and data are saved and zipped in {output_dir_frames}.zip, {output_dir_extrinsics}.zip, and {output_dir_points}.zip')